Georgia Tech OMCS 7643 Deep Learning Spring'24 project

Efficient LLM Fine Tuning
**bold text**

In [2]:
! pip install -q transformers accelerate bitsandbytes xformers datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.4/297.4 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.2/102.2 MB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 222.5/222.5 MB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 48.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 755.5/755.5 MB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 24.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.2.1+cu121 requires torch==2.2.1, but you have torch 2.2.2 which is incompatible.
torchtext 0.17.1 requires torch==2.2.1, but you have torch 2.2.2 which

In [6]:
# load dataset

from datasets import load_dataset
data = load_dataset("glue", "mnli")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/392702 [00:00<?, ? examples/s]

Generating validation_matched split:   0%|          | 0/9815 [00:00<?, ? examples/s]

Generating validation_mismatched split:   0%|          | 0/9832 [00:00<?, ? examples/s]

Generating test_matched split:   0%|          | 0/9796 [00:00<?, ? examples/s]

Generating test_mismatched split:   0%|          | 0/9847 [00:00<?, ? examples/s]

In [15]:
# Prepare training data and define training config

!mkdir offload_folder

from transformers import Trainer, TrainingArguments, AutoModelForSequenceClassification, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("facebook/opt-125m")

model = AutoModelForSequenceClassification.from_pretrained("facebook/opt-125m", num_labels=3)

def manipulate_inputs(inputs):
  return tokenizer(inputs["premise"], inputs["hypothesis"], truncation=True, padding="max_length", max_length=128)

data = data.map(manipulate_inputs, batched=True)

training_args = TrainingArguments(per_device_train_batch_size=8, num_train_epochs=1, output_dir='offload_folder')


mkdir: cannot create directory ‘offload_folder’: File exists


Some weights of OPTForSequenceClassification were not initialized from the model checkpoint at facebook/opt-125m and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/392702 [00:00<?, ? examples/s]

Map:   0%|          | 0/9815 [00:00<?, ? examples/s]

Map:   0%|          | 0/9832 [00:00<?, ? examples/s]

Map:   0%|          | 0/9796 [00:00<?, ? examples/s]

Map:   0%|          | 0/9847 [00:00<?, ? examples/s]

In [17]:
# define trainer object

trainer = Trainer(model=model, args=training_args, train_dataset=data["train"], eval_dataset=data["validation_matched"])

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [10]:
# train the model

trainer.train()

Step,Training Loss
500,0.948100
1000,0.849500
1500,0.806400
2000,0.786300
2500,0.814100
3000,0.790900
3500,0.772400
4000,0.771600
4500,0.766800
5000,0.771100


Step,Training Loss
500,0.948100
1000,0.849500
1500,0.806400
2000,0.786300
2500,0.814100
3000,0.790900
3500,0.772400
4000,0.771600
4500,0.766800
5000,0.771100


TrainOutput(global_step=49088, training_loss=0.6487890895373507, metrics={'train_runtime': 10328.5305, 'train_samples_per_second': 38.021, 'train_steps_per_second': 4.753, 'total_flos': 2.5653170762809344e+16, 'train_loss': 0.6487890895373507, 'epoch': 1.0})

In [11]:
# evaluate the trained model

eval_res = trainer.evaluate()
print(eval_res)

{'eval_loss': 0.5625548958778381, 'eval_runtime': 65.767, 'eval_samples_per_second': 149.239, 'eval_steps_per_second': 18.657, 'epoch': 1.0}


In [18]:
# load validation dataset from mnli dataset
validation_dataset = load_dataset("glue", "mnli", split="validation_matched")
validation_dataset = validation_dataset.map(manipulate_inputs, batched=True)

# Run inference
preds = trainer.predict(validation_dataset)

# model inference
pred_labels = preds.predictions.argmax(axis=1)

print(pred_labels)


[0 1 0 ... 0 2 0]
